## Problem statement:
### Your client is a meal delivery company which operates in multiple cities. They have various fulfillment centers in these cities for dispatching meal orders to their customers. The client wants you to help these centers with demand forecasting for upcoming weeks so that these centers will plan the stock of raw materials accordingly.

### Given the following information, the task is to predict the demand for the next 10 weeks (Weeks: 146-155) for the center-meal combinations in the test set:  

- Historical data of demand for a product-center combination (Weeks: 1 to 145)
- Product(Meal) features such as category, sub-category, current price and discount
- Information for fulfillment center like center area, city information etc.

## Tools and technologies used: 
* Python: Programming language
* Pandas: For data cleaning
* Matplotlib: For data visualisation
* sklearn: For model building

In [22]:
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

## Load dataset

In [23]:
train_data_path = os.path.join("Data", "Raw", "train_data.csv")
test_data_path = os.path.join("Data", "Raw", "test_data.csv")
fullfilment_info_path = os.path.join("Data", "Raw", "fulfilment_center_info.csv")
meal_info_path = os.path.join("Data", "Raw", "meal_info.csv")

train = pd.read_csv(train_data_path)
test = pd.read_csv(test_data_path)
fullfilment_info = pd.read_csv(fullfilment_info_path)
meal_info = pd.read_csv(meal_info_path)

In [24]:
train.head(3)

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189


In [25]:
test.head(3)

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured
0,1028232,146,55,1885,158.11,159.11,0,0
1,1127204,146,55,1993,160.11,159.11,0,0
2,1212707,146,55,2539,157.14,159.14,0,0


In [26]:
fullfilment_info.head(3)

,center_id,city_code,region_code,center_type,op_area
0,11,679,56,TYPE_A,3.7
1,13,590,56,TYPE_B,6.7
2,124,590,56,TYPE_C,4.0


In [27]:
meal_info.head(5)

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian


# Merging datasets to create train data

#### Merge train with fulfillment_info on center_id

In [28]:
train_full=pd.merge(fullfilment_info, train, on='center_id',how='left')
train_full.head(3) 

,center_id,city_code,region_code,center_type,op_area,id,week,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,11,679,56,TYPE_A,3.7,1103215,1,1885,136.83,136.83,0,0,959
1,11,679,56,TYPE_A,3.7,1093939,1,1993,133.89,133.89,0,0,1052
2,11,679,56,TYPE_A,3.7,1120771,1,2539,133.86,135.86,0,0,379


### Merge the result(train_full) with meal_info on meal_id

In [29]:
final_train_df= pd.merge(meal_info, train_full, on='meal_id',how='left')
final_train_df.head(3)

,meal_id,category,cuisine,center_id,city_code,region_code,center_type,op_area,id,week,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1885,Beverages,Thai,11,679,56,TYPE_A,3.7,1103215,1,136.83,136.83,0,0,959
1,1885,Beverages,Thai,11,679,56,TYPE_A,3.7,1091356,2,133.86,135.86,0,0,1054
2,1885,Beverages,Thai,11,679,56,TYPE_A,3.7,1195933,3,135.86,133.86,0,0,891


### Check shape of final train data.

In [30]:
print("Number of rows in train data:",final_train_df.shape[0])
print("Number of columns in train data:",final_train_df.shape[1])

Number of rows in train data: 456548
Number of columns in train data: 15


# Merging datasets to create test data

#### Merge test with fulfillment_info on center_id

In [31]:
test_full=pd.merge(fullfilment_info, test, on='center_id',how='left')
test_full.head(3) 

,center_id,city_code,region_code,center_type,op_area,id,week,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured
0,11,679,56,TYPE_A,3.7,1098228,146,1885,158.17,157.17,0,0
1,11,679,56,TYPE_A,3.7,1101375,146,1993,159.14,159.14,0,0
2,11,679,56,TYPE_A,3.7,1400016,146,2539,159.11,158.11,0,0


### Merge the result(test_full) with meal_info on meal_id

In [32]:
final_test_df=pd.merge(meal_info, test_full, on='meal_id',how='left')
final_test_df.tail(3) 

,meal_id,category,cuisine,center_id,city_code,region_code,center_type,op_area,id,week,checkout_price,base_price,emailer_for_promotion,homepage_featured
32570,1445,Seafood,Continental,51,638,56,TYPE_A,7.0,1004884,153,727.53,726.53,0,0
32571,1445,Seafood,Continental,51,638,56,TYPE_A,7.0,1292684,154,727.53,727.53,0,0
32572,1445,Seafood,Continental,51,638,56,TYPE_A,7.0,1181766,155,678.03,678.03,0,0


### Check shape of final test data.

In [33]:
print("Number of rows in test data:",final_test_df.shape[0])
print("Number of columns in test data:",final_test_df.shape[1])

Number of rows in test data: 32573
Number of columns in test data: 14


# Check column names and data types.

In [34]:
final_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   meal_id                456548 non-null  int64  
 1   category               456548 non-null  object 
 2   cuisine                456548 non-null  object 
 3   center_id              456548 non-null  int64  
 4   city_code              456548 non-null  int64  
 5   region_code            456548 non-null  int64  
 6   center_type            456548 non-null  object 
 7   op_area                456548 non-null  float64
 8   id                     456548 non-null  int64  
 9   week                   456548 non-null  int64  
 10  checkout_price         456548 non-null  float64
 11  base_price             456548 non-null  float64
 12  emailer_for_promotion  456548 non-null  int64  
 13  homepage_featured      456548 non-null  int64  
 14  num_orders             456548 non-nu

In [35]:
final_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32573 entries, 0 to 32572
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   meal_id                32573 non-null  int64  
 1   category               32573 non-null  object 
 2   cuisine                32573 non-null  object 
 3   center_id              32573 non-null  int64  
 4   city_code              32573 non-null  int64  
 5   region_code            32573 non-null  int64  
 6   center_type            32573 non-null  object 
 7   op_area                32573 non-null  float64
 8   id                     32573 non-null  int64  
 9   week                   32573 non-null  int64  
 10  checkout_price         32573 non-null  float64
 11  base_price             32573 non-null  float64
 12  emailer_for_promotion  32573 non-null  int64  
 13  homepage_featured      32573 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 3.5+ 

# Check for missing values

In [36]:
final_train_df.isna().sum()

meal_id                  0
category                 0
cuisine                  0
center_id                0
city_code                0
region_code              0
center_type              0
op_area                  0
id                       0
week                     0
checkout_price           0
base_price               0
emailer_for_promotion    0
homepage_featured        0
num_orders               0
dtype: int64

In [37]:
final_train_df.isna().sum()

meal_id                  0
category                 0
cuisine                  0
center_id                0
city_code                0
region_code              0
center_type              0
op_area                  0
id                       0
week                     0
checkout_price           0
base_price               0
emailer_for_promotion    0
homepage_featured        0
num_orders               0
dtype: int64

### Visualize missing values

In [38]:
# plt.figure(figsize=(10, 5))
# sns.heatmap(final_train_df.isnull(), cmap="viridis", cbar=False, yticklabels=False)
# plt.title("Missing values heatmap of train data")
# plt.show()


In [39]:
# plt.figure(figsize=(10, 5))
# sns.heatmap(final_test_df.isnull(), cmap="viridis", cbar=False, yticklabels=False)
# plt.title("Missing values heatmap of test data")
# plt.show()

# Check for Duplicates

In [ ]:
print("Number of duplicate rows in  test data:", final_train_df.duplicated().sum())

Number of duplicate rows: 0


In [41]:
print("Number of duplicate rows in  train data:", final_test_df.duplicated().sum())

Number of duplicate rows in  train data: 0
